# AMSC698K Homework 4
##### Elijah Kin & Noorain Noorani

In [57]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, AncillaRegister
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import numpy as np


print(qiskit.__version__)

1.2.4


## General Grover's Algorithm Code

In [58]:
# Diffusion (sandwiched reflection) operator
def diffuser(q, n, scratch, label='diffuser'):    
    qc = QuantumCircuit(q)         
    # n = qc.num_qubits   
    qc.h([q[i] for i in range(n-1)])
    qc.x([ q[i] for i in range(n-1) ])         
    if n > 2:
        # for multi-controlled Z use multi-controlled Z rotation
        qc.mcrz(np.pi, q[0:n-2], q[n-2])
    else:
        # for one-qubit oracles no controls
        qc.z(q[0])
    qc.barrier()
    qc.x([ q[i] for i in range(n-1) ])
    qc.h([ q[i] for i in range(n-1) ])    
    return qc

In [134]:
def grover(nqubits, oracle, n_ancilla = 0, iterations=1, measure=True):
    
    q = QuantumRegister(nqubits + n_ancilla, name='q') # create the quantum register
    # aq = AncillaRegister(n_ancilla, name='aq') # create the ancilla quantum register
    if(measure):
    # measure only qubits that are input to the oracle
        creg = ClassicalRegister(nqubits - 1, name='c') 
        # qc = QuantumCircuit(q, aq, creg)         # create the circuit
        qc = QuantumCircuit(q, creg)         # create the circuit
    else:
        qc = QuantumCircuit(q)           # circuit without measurements

    # initialize the circuit:
    # the qubit that receives the oracle output must be set to |1>
    qc.x(nqubits - 1) 
    qc.h([ q[i] for i in range(nqubits - 1) ])  
    qc.h(nqubits - 1)
    qc.save_statevector(label='init')    
    
    # add repetitions of oracle plus diffusion operator
    for i in range(iterations):  
        qc.compose(oracle(q), q, inplace=True)
        qc.barrier()
        qc.compose(diffuser(q, nqubits, n_ancilla), q, inplace=True)
        qc.save_statevector(label=f"diffuser_{i}")

    if(measure):  # Measurements
        # qc.measure(q[0:nqubits+n_ancilla-1],creg) 
        qc.measure(q[0:nqubits-1],creg)          
    return qc

### 1. Use Grover's algorithm (or Amplitude Amplification (QAA)) to solve these problems:

A. Alice, Bob, Charlie, Dora want to set up a meeting. Alice can only meet on Monday, Tuesday, or Thursday; Bob cannot meet on Wednesday; Charlie cannot meet on Thursday, Dora cannot meet on Tuesday nor Friday. Find the day they can meet using Grover's algorithm.

In [ ]:
# Oracle for the Scheduling problem
## Naive Soltuion
def Scheduling_Oracle(q):
    # use a multicontrolled X gate 
    circ = QuantumCircuit(q)
    # TODO
    # Alice's avialble time slots
    circ.mcx([q[0], q[1], q[3]], q[6]) # Available
    circ.mcx([q[2], q[4]], q[6]) # Unavailable
    # Bob's avialble time slots
    circ.mcx([q[0], q[1], q[3], q[4]], q[7])  
    circ.mcx([q[2]], q[7]) 
    # Charlie's avialble time slots
    circ.mcx([q[0], q[1], q[2], q[4]], q[8])
    circ.mcx([q[3]], q[8])
    # Dora's avialble time slots
    circ.mcx([q[0], q[2], q[3]], q[9]) 
    circ.mcx([q[1], q[4]], q[9])

    # Putting together for output
    circ.mcx([q[6], q[7], q[8], q[9]], q[5])
    return circ

# We can construct the oracle using the constraints given in the question
# How would I implement???

In [181]:
nqubits = 6 # 5 for each day of the week and one for the phase kickback

circ_grover1 = grover(nqubits, Scheduling_Oracle, n_ancilla = 4, iterations = 30)
# circ_grover1.draw(output = 'mpl', plot_barriers=True)

In [178]:
# nqubits = 5

# circ_grover1 = grover(nqubits, Scheduling_Oracle, iterations = 1)
# circ_grover1.draw(output = 'mpl', plot_barriers=True)

In [182]:
backend_aer = Aer.get_backend('qasm_simulator')
counts = backend_aer.run(circ_grover1, shots=100).result().get_counts()
print(counts)

{'01000': 1, '11010': 1, '00001': 2, '01011': 3, '10010': 8, '00010': 2, '00011': 3, '00111': 1, '10100': 4, '10011': 4, '01110': 1, '11000': 4, '10101': 5, '11110': 1, '11111': 3, '10110': 6, '01001': 2, '11011': 3, '00100': 6, '01100': 4, '11100': 6, '01010': 3, '00000': 5, '10111': 1, '01101': 6, '01111': 6, '10001': 1, '00110': 1, '11101': 2, '10000': 2, '11001': 3}


B. Alice and Bob work at the front desk of their company from Monday to Wednesday in 2 shifts; each shift is assigned to one employee. Write the constraints as logic expressions and use Grover's algorithm to see whether the shifts can be assigned to Alice and Bob (i) if Alice doesn't want to work more than 4 shifts at the front desk and Bob doesn't want to work on consecutive days at the front desk; (ii) Bob doesn't want to have the same shift on consecutive days and Alice prefers afternoon shifts; what if Alice doesn't want any morning shift?

In [3]:
# TODO

C. Consider a set of 24 marbles, some marbles are white colored and the rest black. Apply QAA to identify the white-colored marbles if 2, 3, 4, or 14 marbles are white-colored. How many sequences of QAA do you have to apply to find the marbles at less than 5% uncertainty.

In [4]:
# TODO

### 2. Given the unsorted list [7,62,48,55,39,17,32,15,27,0,41,3,59,31,51,4].
Use Grover's algorithm (or QAA) to find the list indices of numbers 32, 3, and 27.

In [ ]:
# TODO
# 16 states, 4 qubits
# Want to find 32, 3, and 27.
# Looking for the list indicides
N = 4